In [1]:
# specify the path and then load packages
import sys
sys.path.insert(0, '/Users/timtamothy/Documents/GitHub/adsi_at2/')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data.sets import load_sets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from src.models.pytorch import PytorchDataset

In [2]:
# load our split data
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/ordinal_full/')

In [3]:
X_train

array([[2.484e+03, 4.000e+00, 4.000e+00, 4.000e+00, 4.500e+00],
       [1.700e+03, 4.000e+00, 3.500e+00, 4.000e+00, 5.000e+00],
       [4.266e+03, 4.000e+00, 3.500e+00, 3.500e+00, 3.500e+00],
       ...,
       [4.014e+03, 1.500e+00, 2.500e+00, 1.500e+00, 2.000e+00],
       [4.883e+03, 3.500e+00, 3.500e+00, 3.500e+00, 3.000e+00],
       [2.922e+03, 3.000e+00, 3.500e+00, 3.000e+00, 3.000e+00]])

### Baseline model for our data

In [4]:
# test data on null model
from src.models.null import NullModel

In [5]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

/Users/timtamothy/Documents/GitHub/adsi_at2/src/models/null.py:43: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  self.pred_value = mode(y)[0][0]


In [6]:
from src.models.performance import print_class_perf

In [7]:
print_class_perf(y_base, y_train, set_name = "Training", average = "weighted")

Accuracy Training: 0.009618824683887191
F1 Training: 0.0001832806323283962


### Build Architecture of NN

In [8]:
import torch
import math
print(torch.backends.mps.is_available())

True


In [9]:
print(torch.backends.mps.is_built())

True


In [10]:
from src.models.pytorch import PytorchMultiClass
model = PytorchMultiClass(X_train.shape[1])

In [11]:
device = torch.device('mps')

In [12]:
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=5, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=4, bias=True)
)

In [13]:
# print model architecture
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=5, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=4, bias=True)
)


### Load Data into Model

In [14]:
from torch.utils.data import Dataset, DataLoader
from src.models.pytorch import PytorchDataset

In [15]:
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)

### Train Neural Network

In [16]:
criterion = nn.CrossEntropyLoss()

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [22]:
N_EPOCHS = 30
BATCH_SIZE = 64

In [23]:
from src.models.pytorch import train_classification, test_classification

In [24]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0008	|	Acc: 0.9%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 1
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 2
	(train)	|	Loss: 0.0008	|	Acc: 0.9%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 3
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 4
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 5
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 6
	(train)	|	Loss: 0.0008	|	Acc: 0.9%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 7
	(train)	|	Loss: 0.0008	|	Acc: 0.9%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 8
	(train)	|	Loss: 0.0008	|	Acc: 0.9%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 9
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 10
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 11
	(train)	|	Loss: 0.0008	|	Acc: 1.0%
	(valid)	|	Loss: 0.0008	|	Acc: 1.0%
Epoch: 12
	(train)	|	Loss: